In [1]:
from ruptura.RupturaTable import RupturaTable
from ruptura.CreateBatch import CreateBatch
import collections

In [2]:
rupturaTable = RupturaTable()
files = ['barbieri-only.csv']  #concat all files here
clipData = ['01/11/2018', '01/04/2019']
rupturaTable.generate(files,clipData)

In [ ]:
%%time
createBatch = CreateBatch()
amostras = createBatch.create(rupturaTable.getData(), 'BARBIERI', '01/03/2019')
createBatch.exportBatch(amostras, 'barbieri-tabela-unica-1-3-19.json')

In [ ]:
# TODO - criar um encoder diferente e rodar tudo
# O primeiro vai ser quase igual esta
# O segundo pode ir avancando.

In [ ]:
# TODO (2) - criar os dados para janela deslizante.

In [ ]:
# eu vou vir aqui e definir uma nova forma de gerar esses dados
# seria interessante ter um objeto - data encoder em que ele recebesse a versao e fizesse o encoder correto.
# a primeira que eu quero definir e
# - input - igual estava antes pra comecar
# - saida [ruptura, presenca, desconhecido]

In [ ]:
# Posso colocar presenca 1, reposicao 1/3 e ruptura 0.

# GERANDO TABELA PARA CONFERENCIA

In [ ]:
# APENAS BARBIERI
indBarbieri = []
for i in data.index:
    if data.Produto[i] in todosBiscoitos:
        indBarbieri.append(i)
dataBarbieri = data.iloc[indBarbieri,:].copy()
dataBarbieri = dataBarbieri.reset_index(drop=True)
dataBarbieri.to_csv('data-barbieri.csv',sep=';')

In [27]:
import collections
import json
import numpy as np
import re
import datetime

class CreateBatch: # trocar x por event
    def __init__(self):
        self.PRESENCA = 'Presen a'
        self.RUPTURA_LOJA = 'Ruptura em Loja'
        self.REPOSICAO = 'Reposi o'
        self.RUPTURA_GONDOLA = 'Ruptura em G ndola'
        self.DESCONHECIDO = 'Desconhecido'
        self.SEM_ESTOQUE = 'Sem Unidades Estocadas Armazenadas'
        self.COM_ESTOQUE = 'Unidades Estocadas Armazenadas'
        self.PRODUTO = 'Produto'
        self.OCORRENCIA1 = 'Ocorr ncia'
        self.OCORRENCIA2 = 'Ocorr ncia2'
        self.DATA_OCORRENCIA = 'dataDeOcorrencia'
        self.NOME_CLIENTE = 'Nome do cliente'
        self.TEST_DAYS = 7
        self.titles = []

    def batch(self, fileName, sampleShape = (60,4)):
        amostras = self.loadBatch(fileName)
        print('WARNING - need error handling on samples')
        X = []
        Y = []
        Ytest = []
        for key in amostras:
            x = np.array(amostras[key]['x'])
            y = np.array(amostras[key]['y'])
            yt = np.array(amostras[key]['ytest'])
            if x.shape != sampleShape:
                continue
            X.append(x)
            Y.append(y)
            Ytest.append(yt)
            self.titles.append(key)
        X = np.array(X)
        Y = np.array(Y)
        Ytest = np.array(Ytest)
        return[X, Y, Ytest]

###################################################################################################################    
# FILE HANDLING
###################################################################################################################    
        
    def exportBatch(self, batch, fileName = 'rupturaTable.json'):
        with open(fileName, 'w') as outfile:
            json.dump(batch, outfile)
    
    def loadBatch(self, fileName = 'rupturaTable.json'):
        with open(fileName) as f:
            batchData = json.load(f)
        return batchData

###################################################################################################################    
# CREATE SAMPLES
###################################################################################################################    

    def create(self, data, word, referenceDate):
        allProds = self._searchAllProducts(word, data)
        allSamples = {}
        for prod in allProds:
            sample = self.createItem(data, prod, referenceDate)
            allSamples.update(dict(sample))
        return allSamples

    def createItem(self, data, itemName, referenceDate):
        itemIndexes = self.searchFlag(itemName, data, exactMatch = True)[itemName]
        data = data.loc[itemIndexes,:].copy()
        data = data.reset_index(drop=True)
        # OBJETO ENCODER QUE ABARCA AS DUAS RESPONSABILIDADES ABAIXO
        tableToDictionary = TableToDictionary('0-1-0')
        return tableToDictionary.convertToDict(data, itemName)
        #amostrasItem = self._convertToDict(data, itemName)
        #amostras = self.applyOneHotEncoder(amostrasItem, referenceDate)
        #return amostras

###################################################################################################################    
# SEARCH ITEM IN TABLES
###################################################################################################################    

    def _searchAllProducts(self, word, data):
        products = list(collections.Counter(data.loc[:,self.PRODUTO]))
        allProd  = []
        for prod in products:
            if word in prod:
                allProd.append(prod)
        return allProd
        
    def searchFlag(self, flag, data, columnName = 'Produto', exactMatch = False):
        itemsFlag = {}
        for i in data.index:
            item = data.loc[i,columnName]
            itemEqual = self._itemIsEqual(item, flag, exactMatch)
            if itemEqual:
                if item not in itemsFlag:
                    itemsFlag[item] = [i]
                else:
                    itemsFlag[item].append(i)
        return itemsFlag

    def _itemIsEqual(self, item, flag, exactMatch):
        if exactMatch:
            return flag == item
        else:
            return flag in item

###################################################################################################################    
# TABLE TO DICTIONARY FORMAT
# - key = Nome do cliente-Produto
# - values
#          - x: ocorrencias  (presenca, ruptura, desconhecido e etc.)
#          - data: data da ocorrencia
###################################################################################################################    

    def _convertToDict(self, data, itemName):
        todosOsClientes = list(collections.Counter(data.loc[:,self.NOME_CLIENTE]))
        amostrasItem = {}
        for cliente in todosOsClientes:
            key = self._getKeyName(cliente, itemName)
            amostrasItem[key] = {}
            amostrasItem[key]['x'] = []
            amostrasItem[key]['data'] = []
        for i in data.index:
            cliente = data.loc[i,self.NOME_CLIENTE]
            key = self._getKeyName(cliente, itemName)
            dateI = data.loc[i,self.DATA_OCORRENCIA]
            ocorr = self._getRupturaType(data, i)
            amostrasItem[key]['x'].append(ocorr)
            amostrasItem[key]['data'].append(dateI)
        return amostrasItem

    def _getKeyName(self, cliente, itemName):
        cliente = re.sub('[^A-Za-z0-9]+', ' ', str(cliente))
        itemName = re.sub('[^A-Za-z0-9]+', ' ', str(itemName))
        return str(cliente) + '-' + str(itemName)

    def _getRupturaType(self, data, i):
        if data.loc[i,self.OCORRENCIA1] == self.REPOSICAO:
            if data.loc[i,self.OCORRENCIA2] == self.COM_ESTOQUE:
                return self.RUPTURA_GONDOLA
            else:
                return self.RUPTURA_LOJA
        else:
            return data.loc[i,self.OCORRENCIA1]    

###################################################################################################################    
# ENCODER
###################################################################################################################    
    
    def oneHotEncoder(self, element):
        if element == self.PRESENCA:
            return [1,0,0,0]
        elif element == self.RUPTURA_GONDOLA:
            return [0,1,0,0]
        elif element == self.RUPTURA_LOJA:
            return [0,0,1,0]
        elif element == self.DESCONHECIDO:
            return [0,0,0,1]
        else:
            return 'ELEMENT NOT RECOGNIZED:  ' + str(element)
            
    def decriptEncoder(self, vector):
        if vector == [1,0,0,0]:
            return self.PRESENCA
        elif vector == [0,1,0,0]:
            return self.RUPTURA_GONDOLA
        elif vector == [0,0,1,0]:
            return self.RUPTURA_LOJA
        elif vector == [0,0,0,1]:
            return self.DESCONHECIDO
        else:
            return 'ELEMENT NOT RECOGNIZED:  ' + str(vector)
        
###################################################################################################################    
# APPLY ONE HOT ENCODER
###################################################################################################################    

    def applyOneHotEncoder(self, amostras, referenceDate):
        amostrasTodosClientes = {}
        for cliente in amostras:
            amostraEncoded = self._calculateAmostra(amostras, cliente, referenceDate)
            amostrasTodosClientes.update(amostraEncoded)
        return amostrasTodosClientes

    def _calculateAmostra(self, amostras, cliente, referenceDate):
        x1, x2 = self.defineTrainWindowDates(referenceDate)
        xAmostra = []
        for date in range(x1,x2):
            event = self._getEvent(date,amostras,cliente)
            xAmostra.append(event)
        yAmostra = list(xAmostra)
        yAmostra.pop(0)
        event = self._getEvent(x2,amostras,cliente)
        yAmostra.append(event)
        yTest = []
        for date in range(x2 + 1, x2 + 1 + self.TEST_DAYS):
            event = self._getEvent(date, amostras, cliente)
            yTest.append(event)
        amostraEncoded = self._encodeAmostra(cliente, xAmostra, yAmostra, yTest)
        return amostraEncoded

    def _getEvent(self, date, amostras, cliente):
        if date in amostras[cliente]['data']:
            dateIndex = amostras[cliente]['data'].index(date)
            event = amostras[cliente]['x'][dateIndex]
        else:
            event = self.DESCONHECIDO
        return self.oneHotEncoder(event)

    def _encodeAmostra(self, cliente, xAmostra, yAmostra, yTest):
        amostraEncoded = {}
        amostraEncoded[cliente] = {}
        amostraEncoded[cliente]['x'] = xAmostra
        amostraEncoded[cliente]['y'] = yAmostra
        amostraEncoded[cliente]['ytest'] = yTest
        return amostraEncoded

    
###################################################################################################################    
# CLIP DATES
###################################################################################################################    

    def defineTrainWindowDates(self, referenceDate, window = 60):
        x2 = self._dateToNumber(referenceDate)
        x1 = x2 - window
        return [x1, x2]    

    def _dateToNumber(self, dateI):
        momento = dateI.split('/')
        dia = int(momento[0])
        mes = int(momento[1])
        ano = int(momento[2].split(' ')[0])
        now = datetime.datetime.now()
        return (datetime.datetime(ano, mes, dia) - now).days    



In [28]:
%%time
createBatch = CreateBatch()
amostras = createBatch.createItem(rupturaTable.getData(), 'BISCOITO BARBIERI PAPA OVO 150G', '01/03/2019')
amostras['Mart Minas Paracatu Loja 224-BISCOITO BARBIERI PAPA OVO 150G']

Wall time: 548 ms


In [29]:
amostras

{'Mart Minas Paracatu Loja 224-BISCOITO BARBIERI PAPA OVO 150G': {'x': [None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None],
  'data': [-135,
   -128,
   -124,
   -123,
   -116,
   -113,
   -113,
   -109,
   -107,
   -72,
   -69,
   -66,
   -65,
   -62,
   -60,
   -53,
   -51,
   -48,
   -46,
   -101,
   -100,
   -97,
   -95,
   -93,
   -90,
   -88,
   -86,
   -83,
   -81,
   -80,
   -44,
   -39,
   -37,
   -34,
   -32]},
 'Apoio Betim-BISCOITO BARBIERI PAPA OVO 150G': {'x': [None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
   None,
 

In [26]:
###################################################################################################################    
# TABLE TO DICTIONARY FORMAT
# - key = Nome do cliente-Produto
# - values
#          - x: vetor com infromacoes das medidas referentes naquele dia [presenca, ruptura, e etc.]
#          - data: data da ocorrencia
###################################################################################################################    

import re
import collections

class TableToDictionary:
    def __init__(self, version):
        self._defineTableNames()
        self.__version = version

    def convertToDict(self, data, itemName):
        sampleItem = self._defineSample(data, itemName)
        sampleItem = self._createSampleItem(data, itemName, sampleItem)
        return sampleItem

    def _createSampleItem(self, data, itemName, sampleItem):
        for i in data.index:
            key = self._getKey(data, i, itemName)
            dateI = data.loc[i,self.DATA_OCORRENCIA]
            xValue = self._generateXfromTable(data, i)
            self._addToSample(sampleItem, key, dateI, xValue)
        return sampleItem
    
    def _addToSample(self, sampleItem, key, dateI, xValue):
        sampleItem[key]['x'].append(xValue)
        sampleItem[key]['data'].append(dateI)
    
    def _getKey(self, data, i, itemName):
        client = data.loc[i,self.NOME_CLIENTE]
        return self._getKeyName(client, itemName)
        
    def _getKeyName(self, client, itemName):
        client = re.sub('[^A-Za-z0-9]+', ' ', str(client))
        itemName = re.sub('[^A-Za-z0-9]+', ' ', str(itemName))
        return str(client) + '-' + str(itemName)

    def _generateXfromTable(self, data, i):
        if self.__version[0:4] == '0-0-':
            return self._generateX_V0(data,i)
        if self.__version[0:4] == '0-1-':
            return self._generateX_V1(data,i)
            
    def _generateX_V0(self, data,i): 
        if data.loc[i,self.OCORRENCIA1] == self.REPOSICAO:
            if data.loc[i,self.OCORRENCIA2] == self.COM_ESTOQUE:
                return self.RUPTURA_GONDOLA
            else:
                return self.RUPTURA_LOJA
        else:
            return data.loc[i,self.OCORRENCIA1]
        
    def _generateX_V1(self, data,i):
        x = []
        ocorr = data.loc[i,self.OCORRENCIA1]
        if ocorr == self.RUPTURA_GONDOLA or ocorr == self.RUPTURA_GONDOLA:
            xOcorr = 0
        elif ocorr == self.REPOSICAO:
            xOcorr = 0.33
        else:
            xOcorr = 1
        if data.loc[i,self.OCORRENCIA2] == self.COM_ESTOQUE:
            xEstoque = 1
        else:
            xEstoque = 0
        return [xOcorr, xEstoque]

    def _defineSample(self, data, itemName):
        allClients = list(collections.Counter(data.loc[:,self.NOME_CLIENTE]))
        sampleItem = {}
        for client in allClients:
            key = self._getKeyName(client, itemName)
            sampleItem[key] = {}
            sampleItem[key]['x'] = []
            sampleItem[key]['data'] = []
        return sampleItem
        
    def _defineTableNames(self):
        self.NOME_CLIENTE = 'Nome do cliente'
        self.OCORRENCIA1 = 'Ocorr ncia'
        self.OCORRENCIA2 = 'Ocorr ncia2'
        self.DATA_OCORRENCIA = 'dataDeOcorrencia'
        self.PRESENCA = 'Presen a'
        self.RUPTURA_LOJA = 'Ruptura em Loja'
        self.REPOSICAO = 'Reposi o'
        self.RUPTURA_GONDOLA = 'Ruptura em G ndola'
        self.DESCONHECIDO = 'Desconhecido'
        self.SEM_ESTOQUE = 'Sem Unidades Estocadas Armazenadas'
        self.COM_ESTOQUE = 'Unidades Estocadas Armazenadas'
        self.PRODUTO = 'Produto'
        self.DIA_SEMANA = 'DiaDaSemana'

    
    
        
        

In [18]:
version = '0-0-1'

In [21]:
version[0:4]

'0-0-'

In [ ]:
class Encoder:
    def __init__(self, version):
        self.__version = version
    

###################################################################################################################    
# ENCODER
###################################################################################################################    
    
    def oneHotEncoder(self, element):
        if element == self.PRESENCA:
            return [1,0,0,0]
        elif element == self.RUPTURA_GONDOLA:
            return [0,1,0,0]
        elif element == self.RUPTURA_LOJA:
            return [0,0,1,0]
        elif element == self.DESCONHECIDO:
            return [0,0,0,1]
        else:
            return 'ELEMENT NOT RECOGNIZED:  ' + str(element)
            
    def decriptEncoder(self, vector):
        if vector == [1,0,0,0]:
            return self.PRESENCA
        elif vector == [0,1,0,0]:
            return self.RUPTURA_GONDOLA
        elif vector == [0,0,1,0]:
            return self.RUPTURA_LOJA
        elif vector == [0,0,0,1]:
            return self.DESCONHECIDO
        else:
            return 'ELEMENT NOT RECOGNIZED:  ' + str(vector)
    
        
###################################################################################################################    
# APPLY ONE HOT ENCODER
###################################################################################################################    

    def applyOneHotEncoder(self, amostras, referenceDate):
        amostrasTodosClientes = {}
        for cliente in amostras:
            amostraEncoded = self._calculateAmostra(amostras, cliente, referenceDate)
            amostrasTodosClientes.update(amostraEncoded)
        return amostrasTodosClientes

    def _calculateAmostra(self, amostras, cliente, referenceDate):
        x1, x2 = self.defineTrainWindowDates(referenceDate)
        xAmostra = []
        for date in range(x1,x2):
            event = self._getEvent(date,amostras,cliente) # se tem alguma coisa traz, se nao desconhecido
            xAmostra.append(event)
        yAmostra = list(xAmostra)
        yAmostra.pop(0)
        event = self._getEvent(x2,amostras,cliente)
        yAmostra.append(event)
        yTest = []
        for date in range(x2 + 1, x2 + 1 + self.TEST_DAYS):
            event = self._getEvent(date, amostras, cliente)
            yTest.append(event)
        amostraEncoded = self._encodeAmostra(cliente, xAmostra, yAmostra, yTest)
        return amostraEncoded

    def _getEvent(self, date, amostras, cliente):
        if date in amostras[cliente]['data']:
            dateIndex = amostras[cliente]['data'].index(date)
            event = amostras[cliente]['x'][dateIndex]
        else:
            event = self.DESCONHECIDO
        return self.oneHotEncoder(event)

    def _encodeAmostra(self, cliente, xAmostra, yAmostra, yTest):
        amostraEncoded = {}
        amostraEncoded[cliente] = {}
        amostraEncoded[cliente]['x'] = xAmostra
        amostraEncoded[cliente]['y'] = yAmostra
        amostraEncoded[cliente]['ytest'] = yTest
        return amostraEncoded

    
###################################################################################################################    
# CLIP DATES
###################################################################################################################    

    def defineTrainWindowDates(self, referenceDate, window = 60):
        x2 = self._dateToNumber(referenceDate)
        x1 = x2 - window
        return [x1, x2]    

    def _dateToNumber(self, dateI):
        momento = dateI.split('/')
        dia = int(momento[0])
        mes = int(momento[1])
        ano = int(momento[2].split(' ')[0])
        now = datetime.datetime.now()
        return (datetime.datetime(ano, mes, dia) - now).days    

